In [1]:
import pandas as pd
import numpy as np
from Project.Database import Db
from Project._03FeatureSelection.NZERTF_dataframe_redundancy_functions import find_average_power_consumption_per_minute
from Project._05InferKnowledgeOfRules.infer_rules_functions import SE_time_df, json_to_dataframe

In [2]:
app_stats = SE_time_df(json_to_dataframe(year=1, level=1, with_redundancy=False))
app_stats

{'Load_StatusRefrigerator':     TotalAbsSupport  AbsSupport  EventCount  ExternalUtility  RelSupport  \
 0               598         362       13185              1.0    0.991781   
 1               699         361       13185              1.0    0.989041   
 2               704         360       13185              1.0    0.986301   
 3               693         361       13185              1.0    0.989041   
 4               697         361       13185              1.0    0.989041   
 5               687         361       13185              1.0    0.989041   
 6               689         360       13185              1.0    0.986301   
 7               692         361       13185              1.0    0.989041   
 8               697         362       13185              1.0    0.991781   
 9               693         362       13185              1.0    0.991781   
 10              702         361       13185              1.0    0.989041   
 11              701         361       13185     

In [3]:
average_power_consumption_per_minute_df = find_average_power_consumption_per_minute()
average_power_consumption_per_minute_df

,PowerConsumption(Wm)
Load_StatusApplianceCooktop,1496.0
Load_StatusApplianceDishwasher,282.0
Load_StatusApplianceOven,771.0
Load_StatusApplianceRangeHood,78.0
Load_StatusBA1Lights,48.0
Load_StatusBA2Lights,11.0
Load_StatusBR2Lights,40.0
Load_StatusBR3Lights,33.0
Load_StatusBR4Lights,27.0
Load_StatusDRLights,38.0


In [43]:
matrix_dict={}
for key, value in app_stats.items():
    lifespan = value['Lifespan'][0]
    hour_slots = int(np.ceil(lifespan/60))

    average_power_consumption_per_minute = average_power_consumption_per_minute_df.loc[lambda self: self.index == key].values[0][0]

    hour_list = [0 for n in range(24)]
    for i in range(hour_slots):
        hour_list[i] = round(average_power_consumption_per_minute*lifespan/60/hour_slots, 2)

    temp_dict={}
    for n in range(24+1-hour_slots):
        rotate_list = [hour_list[(i - n) % len(hour_list)] for i, x in enumerate(hour_list)]
        temp_dict.update({n: rotate_list})

    matrix_dict.update({key: pd.DataFrame(temp_dict).loc[:,lambda self: self.sum(0) == self.T.dot(app_stats[key]['TimeAssociation'])]})

    # matrix_dict[key] = matrix_dict[key]

matrix_dict

{'Load_StatusRefrigerator':        0      1      2      3      4      5      6      7      8      9   ...  \
 0   35.67   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00  ...   
 1    0.00  35.67   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00  ...   
 2    0.00   0.00  35.67   0.00   0.00   0.00   0.00   0.00   0.00   0.00  ...   
 3    0.00   0.00   0.00  35.67   0.00   0.00   0.00   0.00   0.00   0.00  ...   
 4    0.00   0.00   0.00   0.00  35.67   0.00   0.00   0.00   0.00   0.00  ...   
 5    0.00   0.00   0.00   0.00   0.00  35.67   0.00   0.00   0.00   0.00  ...   
 6    0.00   0.00   0.00   0.00   0.00   0.00  35.67   0.00   0.00   0.00  ...   
 7    0.00   0.00   0.00   0.00   0.00   0.00   0.00  35.67   0.00   0.00  ...   
 8    0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00  35.67   0.00  ...   
 9    0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00  35.67  ...   
 10   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.